In [4]:
#nuevo enfoque
# --- Importaciones para Manejo de Archivos y Directorios ---
import os
import sys
import subprocess
import re
import math
# --- Importamos Vertex AI ---
import vertexai
# --- Importaciones para Visualización y Formato de Texto ---
# Se utiliza para mostrar contenido enriquecido, como texto con formato Markdown,
# directamente en entornos como Jupyter Notebooks o IPython.
from IPython.display import Markdown, display
# Importa la clase `Markdown` de la biblioteca `rich`, que sirve para renderizar
# Markdown con formato avanzado en la terminal. Se le da un alias `RichMarkdown`
# para evitar conflictos de nombre con la importación anterior.
from rich.markdown import Markdown as RichMarkdown
# --- Importaciones para el Modelo Generativo de Vertex AI ---
# Importa las clases necesarias del SDK de Vertex AI para interactuar con los modelos generativos.
# - GenerationConfig: Para configurar los parámetros de la respuesta (ej. temperatura, top_p).
# - GenerativeModel: La clase principal para cargar y usar un modelo generativo como Gemini.
# - Image: Para manejar y enviar imágenes como parte de la entrada al modelo (enfoque multimodal).
from vertexai.generative_models import GenerationConfig, GenerativeModel, Image

In [3]:
# --- Para el proceso de datos y visualización ---
import numpy as np
import pandas as pd
import json
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
import time
# --- Para manejar las variables de entorno ---
import os
from dotenv import load_dotenv
# --- Desactiva las advertencias de asignaciones encadenadas en pandas para evitar mensajes de warning al modificar DataFrames.
pd.options.mode.chained_assignment = None  # default='warn'
# --- Dependencias de Vertex AI ---
import vertexai                                              # Importa el módulo principal de Vertex AI.
from vertexai import init                                    # Inicializa Vertex AI con las credenciales y configuraciones necesarias.
from vertexai.vision_models import Image as VMImage          # Importa la clase Image de Vertex AI para manejar imágenes.
from vertexai.vision_models import MultiModalEmbeddingModel  # Importa el modelo de embeddings multimodales de Vertex AI para procesar imágenes y videos.
from vertexai.vision_models import Video                     # Clase para manejar archivos de video en Vertex AI.
from vertexai.vision_models import VideoSegmentConfig        # Configuración para segmentar videos
from vertexai.generative_models import GenerativeModel       # Importa la clase para modelos generativos, como Gemini.
from vertexai.generative_models import Part                  # Importa la clase Part para manejar partes de un mensaje, como texto o imágenes.
# --- Para conectarse y consultar un endpoint de búsqueda vectorial (Vector Search) en Vertex AI. 
from google.cloud.aiplatform.matching_engine import MatchingEngineIndexEndpoint 
# --- Para acceder a los buckets de Google Cloud Storage y manejar archivos.
from google.cloud import storage
# --- Dependencias para poder visualizar ---
from IPython.display import Video as MVideo                  # Permite mostrar videos directamente en celdas de Jupyter Notebook.
from IPython.display import HTML                             # Permite mostrar contenido HTML en celdas de Jupyter Notebook.
from IPython.display import Image as ImageByte               # Permite mostrar imágenes en el notebook (renombrado como ImageByte para evitar conflictos de nombres).
from IPython.display import display                          # Función general para mostrar objetos en el notebook.
from sklearn.metrics.pairwise import cosine_similarity       # Función para calcular la similitud coseno entre vectores, útil para comparar embeddings.

In [17]:
from google.cloud import storage

def listar_objetos_en_carpeta(nombre_bucket: str, nombre_carpeta: str) -> list[str]:
    """
    Lista los nombres de todos los objetos (archivos) dentro de una carpeta específica
    en un bucket de Google Cloud Storage.

    Args:
        nombre_bucket: El nombre de tu bucket de GCS (ej. 'vboxiooof').
        nombre_carpeta: La ruta de la carpeta dentro del bucket (ej. 'videos/').

    Returns:
        Una lista de strings con los nombres completos de los objetos.
    """
    # Inicializa el cliente de GCS. Asegúrate de que tus credenciales
    # estén configuradas en el entorno donde ejecutes el código.
    storage_client = storage.Client()
    
    # Obtiene el objeto bucket.
    bucket = storage_client.get_bucket(nombre_bucket)

    # Usa el parámetro 'prefix' para filtrar los objetos por la carpeta.
    # El 'prefix' es la ruta de la carpeta, por ejemplo, 'videos/'.
    # Si quieres listar solo los archivos de video, puedes agregar un filtro adicional.
    blobs = bucket.list_blobs(prefix=nombre_carpeta)

    nombres_objetos = []
    print(f"Buscando objetos en la carpeta '{nombre_carpeta}' del bucket '{nombre_bucket}'...")
    
    for blob in blobs:
        # Aquí, 'blob.name' te da la ruta completa del objeto, como 'videos/mi_video.mp4'.
        # Puedes añadir una condición para ignorar la propia carpeta si está listada.
        if blob.name != nombre_carpeta:
            nombres_objetos.append(blob.name)
            print(f"  - Objeto encontrado: {blob.name}")
            
    print(f"\nSe encontraron {len(nombres_objetos)} objetos.")
    return nombres_objetos

# --- Ejemplo de uso ---
if __name__ == "__main__":
    tu_bucket = "vboxiooof"
    tu_carpeta = "Videos/pruebas/"  # Asegúrate de incluir la barra diagonal al final

    # Llama a la función para obtener la lista de objetos.
    lista_de_archivos = listar_objetos_en_carpeta(tu_bucket, tu_carpeta)

    # Puedes imprimir la lista completa si lo deseas
    # print("\nLista completa de archivos:", lista_de_archivos)
    
    # Esta lista de archivos la puedes usar para generar las URIs
    # 'gs://bucket_name/object_name' para enviárselas a Gemini.

Buscando objetos en la carpeta 'Videos/pruebas/' del bucket 'vboxiooof'...
  - Objeto encontrado: Videos/pruebas/mexicosta_segment_001.mkv
  - Objeto encontrado: Videos/pruebas/mexicosta_segment_002.mkv
  - Objeto encontrado: Videos/pruebas/mexicosta_segment_003.mkv
  - Objeto encontrado: Videos/pruebas/mexicosta_segment_004.mkv
  - Objeto encontrado: Videos/pruebas/mexicosta_segment_005.mkv
  - Objeto encontrado: Videos/pruebas/mexicosta_segment_018.mkv
  - Objeto encontrado: Videos/pruebas/mexicosta_segment_021.mkv
  - Objeto encontrado: Videos/pruebas/mexicosta_segment_022.mkv
  - Objeto encontrado: Videos/pruebas/mexicosta_segment_024.mkv
  - Objeto encontrado: Videos/pruebas/mexicosta_segment_257.mkv
  - Objeto encontrado: Videos/pruebas/mexicosta_segment_258.mkv
  - Objeto encontrado: Videos/pruebas/mexicosta_segment_259.mkv
  - Objeto encontrado: Videos/pruebas/mexicosta_segment_274.mkv
  - Objeto encontrado: Videos/pruebas/mexicosta_segment_289.mkv
  - Objeto encontrado: Videos

In [25]:
#este ya me da el analisis de los primeros 5 videos


from google.cloud import storage
from vertexai.preview.generative_models import GenerativeModel, Part
import time
from typing import List, Dict

def analizar_primeros_5_videos(
    nombre_bucket: str, 
    carpeta_videos: str, 
    pregunta_usuario: str
) -> Dict[str, str]:
    """
    Analiza solo los primeros 5 videos de un bucket con Gemini.
    
    Args:
        nombre_bucket: Nombre del bucket de GCS
        carpeta_videos: Ruta de la carpeta con videos
        pregunta_usuario: Pregunta para analizar los videos
        
    Returns:
        Resultados del análisis por video (solo primeros 5)
    """
    # 1. Listar los videos en el bucket
    print("Listando videos en el bucket...")
    todos_videos = listar_objetos_en_carpeta(nombre_bucket, carpeta_videos)
    
    # Filtrar solo archivos de video y tomar primeros 5
    video_extensions = ('.mp4', '.mov', '.avi', '.mkv')
    videos = [v for v in todos_videos if v.lower().endswith(video_extensions)][:5]
    
    if not videos:
        print("No se encontraron videos para analizar.")
        return {}
    
    print(f"\nSe analizarán los primeros {len(videos)} videos con Gemini...")
    for i, video in enumerate(videos, 1):
        print(f"{i}. {video}")
    
    # 2. Analizar cada video con Gemini
    resultados = analizar_videos_con_gemini(nombre_bucket, videos, pregunta_usuario)
    
    return resultados



def analizar_videos_con_gemini(
    nombre_bucket: str, 
    nombres_videos: List[str], 
    marca_usuario: str,
    max_retries: int = 2,
    delay: float = 5.0
) -> Dict[str, str]:
    """
    Analiza videos buscando una marca específica con instrucciones optimizadas para mejor detección.
    """
    model = GenerativeModel('gemini-2.5-pro')
    resultados = {}
    
    for video_path in nombres_videos:
        gcs_uri = f"gs://{nombre_bucket}/{video_path}"
        print(f"\nAnalizando video: {video_path}")
        
        # Prompt para detección de marcas
        prompt = f"""
        Eres un especialista en análisis de contenido deportivo con capacidad avanzada para detectar 
        marcas comerciales en videos. Sigue estrictamente estas instrucciones:

        1. BUSCAR ESPECÍFICAMENTE y ÚNICAMENTE la marca: {marca_usuario}
        2. Analizar TODO el video frame por frame si es necesario
        3. Considerar estas formas de aparición:
           
           - Menciones verbales (en comentarios o anuncios)
           - Tapetes al lado de porterías
           - Vallas electronicas (logo o nombre de la marca)
           - Marcador (que se encuentra al lado del marcador)
           - Banner (anuncios rectangulares, no cubren toda la pantalla, tienden a aparecer en la parte inferior o superior o a los lados de la pantalla, no tienen transparencia))
           - Comercial (solo si salen personas hablando de la marca o interactuando entre si)
           - Virtual (tipo de anuncio que deja ver la cancha por detras, pueden ser animaciones, alineaciones del partido, deja entrever lo que sucede detras en la cancha o publico, puede ser el logo en el centro de la cancha real o animada)


        4. Para CADA detección proveer:
           - TIEMPO INICIO: (formato HH:MM:SS)
           - DURACIÓN: (segundos)
           - TIPO DE MEDIO: (seleccionar de lista anterior)
           - TAMAÑO: (pequeño <10% pantalla, mediano 10-30%, grande >30%)
           - UBICACIÓN: (esquina superior derecha, centro, etc.)
           - MENCION VERBAL: (si aplica, con tiempo exacto)

        5. Si NO se detecta la marca:
           - Indicar "Marca no encontrada después de análisis completo"
           - No dar detalles adicionales

        6. Formato de respuesta obligatorio:
        [INICIO ANÁLISIS]
        Video: {video_path}
        Marca buscada: {marca_usuario}
        ---
        Detecciones:
        [Detección 1]
        Tiempo: HH:MM:SS - HH:MM:SS
        Duración: X segundos
        Tipo: [TIPO DE MEDIO]
        Tamaño: [TAMAÑO]
        Ubicación: [UBICACIÓN]
        Contexto: [DESCRIPCIÓN]
        Mencionada: [SI/NO] (HH:MM:SS si aplica)
        ---
        [Detección N]...
        ---
        Resumen: Total de apariciones: X
        [FIN ANÁLISIS]

        IMPORTANTE: Si el video es de baja calidad o la marca aparece muy breve, 
        igualmente reportarla si es identificable.
        Si no es muy visible, pasarla por alto.
        """
        
        for attempt in range(max_retries):
            try:
                response = model.generate_content(
                    [
                        Part.from_uri(gcs_uri, mime_type="video/mp4"),
                        Part.from_text(prompt)
                    ],
                    generation_config={
                        "temperature": 0.3,  # Más preciso que creativo
                        "max_output_tokens": 2000
                    }
                )
                
                # Procesar respuesta para asegurar formato
                respuesta = response.text
                if marca_usuario.lower() not in respuesta.lower():
                    respuesta = f"Marca '{marca_usuario}' no encontrada después de análisis exhaustivo\n\n{respuesta}"
                
                resultados[video_path] = respuesta
                print(f"Análisis completado para {video_path}")
                break
                
            except Exception as e:
                print(f"Intento {attempt + 1} fallido para {video_path}: {str(e)}")
                if attempt == max_retries - 1:
                    resultados[video_path] = f"Error en análisis: {str(e)}"
                else:
                    time.sleep(delay)
    
    return resultados


# Ejemplo de uso actualizado
if __name__ == "__main__":
    # Configuración
    BUCKET = "vboxiooof"
    CARPETA = "Videos/pruebas/"
    MARCA = "caliente.mx"  # El usuario solo necesita proporcionar la marca
    
    # Ejecutar análisis
    resultados = analizar_primeros_5_videos(BUCKET, CARPETA, MARCA)
    
    # Mostrar resultados
    print(f"\n\n--- Resultados del análisis para la marca '{MARCA}' ---")
    for video, respuesta in resultados.items():
        print(f"\nVideo: {video}")
        print(f"Reporte de apariciones:\n{respuesta}\n")
        print("-" * 50)

Listando videos en el bucket...
Buscando objetos en la carpeta 'Videos/pruebas/' del bucket 'vboxiooof'...
  - Objeto encontrado: Videos/pruebas/mexicosta_segment_001.mkv
  - Objeto encontrado: Videos/pruebas/mexicosta_segment_021.mkv
  - Objeto encontrado: Videos/pruebas/mexicosta_segment_022.mkv
  - Objeto encontrado: Videos/pruebas/mexicosta_segment_024.mkv
  - Objeto encontrado: Videos/pruebas/mexicosta_segment_257.mkv
  - Objeto encontrado: Videos/pruebas/mexicosta_segment_258.mkv
  - Objeto encontrado: Videos/pruebas/mexicosta_segment_259.mkv
  - Objeto encontrado: Videos/pruebas/mexicosta_segment_274.mkv
  - Objeto encontrado: Videos/pruebas/mexicosta_segment_289.mkv
  - Objeto encontrado: Videos/pruebas/mexicosta_segment_290.mkv
  - Objeto encontrado: Videos/pruebas/mexicosta_segment_295.mkv
  - Objeto encontrado: Videos/pruebas/mexicosta_segment_296.mkv
  - Objeto encontrado: Videos/pruebas/mexicosta_segment_301.mkv
  - Objeto encontrado: Videos/pruebas/mexicosta_segment_302.m

/Users/wivboost/Documents/Repositorio/Embeddings_OFF/mi-entorno/lib/python3.12/site-packages/vertexai/generative_models/_generative_models.py:433: UserWarning: This feature is deprecated as of June 24, 2025 and will be removed on June 24, 2026. For details, see https://cloud.google.com/vertex-ai/generative-ai/docs/deprecations/genai-vertexai-sdk.
  warning_logs.show_deprecation_warning()


Análisis completado para Videos/pruebas/mexicosta_segment_001.mkv

Analizando video: Videos/pruebas/mexicosta_segment_021.mkv
Análisis completado para Videos/pruebas/mexicosta_segment_021.mkv

Analizando video: Videos/pruebas/mexicosta_segment_022.mkv
Análisis completado para Videos/pruebas/mexicosta_segment_022.mkv

Analizando video: Videos/pruebas/mexicosta_segment_024.mkv
Análisis completado para Videos/pruebas/mexicosta_segment_024.mkv

Analizando video: Videos/pruebas/mexicosta_segment_257.mkv
Análisis completado para Videos/pruebas/mexicosta_segment_257.mkv


--- Resultados del análisis para la marca 'caliente.mx' ---

Video: Videos/pruebas/mexicosta_segment_001.mkv
Reporte de apariciones:
[INICIO ANÁLISIS]
Video: Videos/pruebas/mexicosta_segment_001.mkv
Marca buscada: caliente.mx
---
Detecciones:
[Detección 1]
Tiempo: 00:00:02 - 00:00:06
Duración: 4 segundos
Tipo: Vallas electronicas
Tamaño: Mediano
Ubicación: Superior
Contexto: La marca aparece de forma repetida y clara en las 